※推論用データではない 候補1 実力テスト(クラス分けやらず・3σ法・ハイパーパラメータ調整).ipynb のコピー のコピー
…より、

"/content/submission (推論データではない).csv"

を提出したが、推論データではなく、テスト用データだった。

解決：y_pred = モデル名.predict(推論用データ)

のうち、入力値である推論用データのndarrayの形が(サンプルサイズ、各サンプルの次元数)となるようにする。

エラー：Found input variables with inconsistent numbers of samples: [10000, 1192]

In [140]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

%matplotlib inline
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

from sklearn.preprocessing import StandardScaler
std_scaler = StandardScaler()

from sklearn.linear_model import LinearRegression
model = LinearRegression()

from sklearn import metrics

In [141]:
df_test = pd.read_csv('/content/bank_test.csv')
df_train = pd.read_csv('/content/bank_train.csv')
df_pred = pd.read_csv('/content/bank_pred.csv')
df_train.head(3)

,age,job,marital,education,default,balance,housing,loan,pdays,previous,poutcome,result
0,50.0,entrepreneur,married,secondary,no,1857,no,yes,-1.0,0,unknown,fail
1,37.0,technician,single,secondary,no,363,yes,no,-1.0,0,unknown,fail
2,35.0,blue-collar,married,primary,no,422,yes,no,-1.0,0,unknown,fail


In [142]:
df_test.head(3)

,age,job,marital,education,default,balance,housing,loan,pdays,previous,poutcome,result
0,55.0,self-employed,married,tertiary,no,1810,yes,no,-1.0,0,unknown,fail
1,54.0,management,married,tertiary,no,364,yes,no,-1.0,0,unknown,fail
2,35.0,technician,married,secondary,no,-180,no,yes,-1.0,0,unknown,success


In [143]:
df_pred.head(3)

,age,job,marital,education,default,balance,housing,loan,pdays,previous,poutcome
0,53,services,married,secondary,no,1364,no,no,-1,0,unknown
1,36,blue-collar,married,secondary,no,1949,yes,no,319,3,failure
2,40,admin.,married,unknown,no,3557,yes,no,-1,0,unknown


In [144]:
print(df_train.shape)
print(df_test.shape)
print(df_pred.shape)

(6416, 12)
(15000, 12)
(10000, 11)


#重複行削除

In [145]:
#重複行処理前
df_train.duplicated(keep=False).value_counts()

False    6357
True       59
dtype: int64

In [146]:
df_train = df_train.drop_duplicates()
#重複行処理後
df_train.duplicated(keep=False).value_counts()

False    6384
dtype: int64

In [147]:
#重複行処理前
#df_pred.duplicated(keep=False).value_counts()

In [148]:
#df_pred = df_pred.drop_duplicates()
#重複行処理後
#df_pred.duplicated(keep=False).value_counts()

In [149]:
print(df_train.shape)
print(df_test.shape)
print(df_pred.shape)

(6384, 12)
(15000, 12)
(10000, 11)


#欠損値処理1(列を全部消す)

In [150]:
#axis=1なので、列方向に削除が実行されます。
df_train = df_train.drop(labels='poutcome', axis=1)
df_train = df_train.drop(labels='loan', axis=1)

df_pred = df_pred.drop(labels='poutcome', axis=1)
df_pred = df_pred.drop(labels='loan', axis=1)

In [151]:
print(df_train.shape)
print(df_test.shape)
print(df_pred.shape)

(6384, 10)
(15000, 12)
(10000, 9)


#欠損値処理2(外れ値を平均値で補完)

In [152]:
# 平均値の確認
print(df_train['age'].mean())
print(df_train['balance'].mean())
print(df_train['pdays'].mean())
print(df_train['previous'].mean())

40.99555131871624
1336.2005012531329
39.821203528670445
0.5762844611528822


In [153]:
# 欠損値を平均値で補完
df_train = df_train.fillna({'age':df_train['age'].mean()})
df_train = df_train.fillna({'balance':df_train['balance'].mean()})
df_train = df_train.fillna({'pdays':df_train['pdays'].mean()})
df_train = df_train.fillna({'prevous':df_train['previous'].mean()})

In [154]:
print(df_train.shape)
print(df_test.shape)
print(df_pred.shape)

(6384, 10)
(15000, 12)
(10000, 9)


#カテゴリカル変数処理(One Hot Encording)

In [155]:
# 最頻値を使用して欠損値を補完
df_train = df_train.fillna({'job':df_train['job'].mode()[0]})
df_train = df_train.fillna({'marital':df_train['marital'].mode()[0]})
df_train = df_train.fillna({'default':df_train['default'].mode()[0]})
df_train = df_train.fillna({'housing':df_train['housing'].mode()[0]})

In [156]:
# 最頻値を使用して欠損値を補完
df_pred = df_pred.fillna({'job':df_pred['job'].mode()[0]})
df_pred = df_pred.fillna({'marital':df_pred['marital'].mode()[0]})
df_pred = df_pred.fillna({'default':df_pred['default'].mode()[0]})
df_pred = df_pred.fillna({'housing':df_pred['housing'].mode()[0]})

In [157]:
print(df_train.shape)
print(df_test.shape)
print(df_pred.shape)

(6384, 10)
(15000, 12)
(10000, 9)


#特徴量変換(Label Encording)

In [158]:
# カテゴリカル変数を含んだデータのみを抽出
df_train_obj = df_train.select_dtypes(include='object')
df_train_uni = df_train_obj.nunique()

In [159]:
# ユニークな値が 2 種類の列名のみ取得
df_cols = df_train_uni[df_train_uni == 2].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_train[col])
  df_train[col] = le.transform(df_train[col])

In [160]:
# ユニークな値が 4 種類の列名のみ取得
df_cols = df_train_uni[df_train_uni == 4].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_train[col])
  df_train[col] = le.transform(df_train[col])

In [161]:
# ユニークな値が 3 種類の列名のみ取得
df_cols = df_train_uni[df_train_uni == 3].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_train[col])
  df_train[col] = le.transform(df_train[col])

In [162]:
# ユニークな値が 3 種類の列名のみ取得
df_cols = df_train_uni[df_train_uni == 12].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_train[col])
  df_train[col] = le.transform(df_train[col])

In [163]:
print(df_train.shape)
print(df_test.shape)
print(df_pred.shape)

(6384, 10)
(15000, 12)
(10000, 9)


In [164]:
# カテゴリカル変数を含んだデータのみを抽出
df_pred_obj = df_pred.select_dtypes(include='object')
df_pred_uni = df_pred_obj.nunique()

In [165]:
# ユニークな値が 2 種類の列名のみ取得
df_cols = df_pred_uni[df_pred_uni == 2].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_pred[col])
  df_pred[col] = le.transform(df_pred[col])

In [166]:
# ユニークな値が 3 種類の列名のみ取得
df_cols = df_pred_uni[df_pred_uni == 3].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_pred[col])
  df_pred[col] = le.transform(df_pred[col])

In [167]:
# ユニークな値が 4 種類の列名のみ取得
df_cols = df_pred_uni[df_pred_uni == 4].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_pred[col])
  df_pred[col] = le.transform(df_pred[col])

In [168]:
# ユニークな値が 12 種類の列名のみ取得
df_cols = df_pred_uni[df_pred_uni == 12].index

for col in df_cols:
  le = LabelEncoder()
  le.fit(df_pred[col])
  df_pred[col] = le.transform(df_pred[col])

In [169]:
print(df_train.shape)
print(df_test.shape)
print(df_pred.shape)

(6384, 10)
(15000, 12)
(10000, 9)


#外れ値処理（3σ法）

In [170]:
df_train.describe()

,age,job,marital,education,default,balance,housing,pdays,previous,result
count,6384.000000,6384.000000,6384.000000,6384.000000,6384.000000,6384.000000,6384.000000,6384.000000,6384.000000,6384.000000
mean,40.995551,4.276316,1.164160,1.228227,0.019737,1336.200501,0.555138,39.821204,0.576284,0.114818
std,10.547951,3.257034,0.601575,0.748540,0.139105,2716.573432,0.496989,97.672046,1.959517,0.318828
min,18.000000,0.000000,0.000000,0.000000,0.000000,-3313.000000,0.000000,-1.000000,0.000000,0.000000
25%,33.000000,1.000000,1.000000,1.000000,0.000000,78.000000,0.000000,-1.000000,0.000000,0.000000
50%,39.000000,4.000000,1.000000,1.000000,0.000000,477.500000,1.000000,-1.000000,0.000000,0.000000
75%,48.000000,7.000000,2.000000,2.000000,0.000000,1472.000000,1.000000,-1.000000,0.000000,0.000000
max,93.000000,11.000000,2.000000,3.000000,1.000000,43074.000000,1.000000,838.000000,55.000000,1.000000


In [171]:
mu = df_train['housing'].mean() # 平均値
sigma = df_train['housing'].std() # 標準偏差

In [172]:
df_train3 = df_train[(mu - 3 * sigma <= df_train['housing']) & (df_train['housing'] <= mu + 3 * sigma)]

In [173]:
mu = df_train['pdays'].mean() # 平均値
sigma = df_train['pdays'].std() # 標準偏差

In [174]:
df_train3 = df_train[(mu - 3 * sigma <= df_train['pdays']) & (df_train['pdays'] <= mu + 3 * sigma)]

In [175]:
mu = df_train['balance'].mean() # 平均値
sigma = df_train['balance'].std() # 標準偏差

In [176]:
df_train3 = df_train[(mu - 3 * sigma <= df_train['balance']) & (df_train['balance'] <= mu + 3 * sigma)]

In [124]:
mu = df_train['previous'].mean() # 平均値
sigma = df_train['previous'].std() # 標準偏差

In [125]:
df_train3 = df_train[(mu - 3 * sigma <= df_train['previous']) & (df_train['previous'] <= mu + 3 * sigma)]

In [177]:
print(df_train3.shape)
print(df_test.shape)
print(df_pred.shape)

(6267, 10)
(15000, 12)
(10000, 9)


#ハイパーパラメータ調整(Grid Search)

In [178]:
# 入力変数と出力変数の切り分け
t = df_train3['result'].values
x = df_train3.drop(['result'], axis=1).values

# 学習データとテストデータの切り分け
from sklearn.model_selection import train_test_split
x_train_val, x_test, t_train_val, t_test = train_test_split(x, t, test_size=0.2, random_state=1)

# 検証用データセット：学習用データセット＝ 30 ： 70
x_train, x_val, t_train, t_val = train_test_split(x_train_val, t_train_val, test_size=0.3, random_state=1)

In [179]:
x_train.shape, x_val.shape, x_test.shape

((3509, 9), (1504, 9), (1254, 9))

In [180]:
# GridSearchCV クラスのインポート
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier

# 学習に使用するアルゴリズムの定義
estimator = DecisionTreeClassifier(random_state=0)

# 探索するハイパーパラメータと範囲の定義
param_grid = [{
    'max_depth': [3, 20, 50],
    'min_samples_split': [3, 20, 30]
}]

# データセット分割数を定義
cv = 5

# GridSearchCV クラスを用いたモデルの定義
tuned_model = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=cv, return_train_score=False)

# モデルの学習＆検証
tuned_model.fit(x_train_val, t_train_val)

# 検証結果の確認
pd.DataFrame(tuned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.005118,0.004133,0.004051,0.014224,0.0153,0.014288,0.014936,0.017427,0.013644
std_fit_time,0.001086,0.000053,0.000035,0.000432,0.001448,0.001325,0.000626,0.003916,0.000611
mean_score_time,0.000887,0.00074,0.000656,0.000999,0.001059,0.000989,0.00101,0.001198,0.001
std_score_time,0.0003,0.000127,0.000016,0.000038,0.000207,0.000059,0.000067,0.000272,0.000053
param_max_depth,3,3,3,20,20,20,50,50,50
param_min_samples_split,3,20,30,3,20,30,3,20,30
params,"{'max_depth': 3, 'min_samples_split': 3}","{'max_depth': 3, 'min_samples_split': 20}","{'max_depth': 3, 'min_samples_split': 30}","{'max_depth': 20, 'min_samples_split': 3}","{'max_depth': 20, 'min_samples_split': 20}","{'max_depth': 20, 'min_samples_split': 30}","{'max_depth': 50, 'min_samples_split': 3}","{'max_depth': 50, 'min_samples_split': 20}","{'max_depth': 50, 'min_samples_split': 30}"
split0_test_score,0.88335,0.88335,0.88335,0.845464,0.865404,0.865404,0.836491,0.861416,0.865404
split1_test_score,0.889332,0.889332,0.889332,0.832502,0.874377,0.880359,0.825523,0.871386,0.879362
split2_test_score,0.890329,0.890329,0.890329,0.844467,0.878365,0.877368,0.837488,0.878365,0.877368


In [181]:
# GridSearchCV クラスのインポート
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier

# 学習に使用するアルゴリズムの定義
estimator = DecisionTreeClassifier(random_state=0)

# 探索するハイパーパラメータと範囲の定義
param_grid = [{
    'max_depth': [3, 10, 50],
    'min_samples_split': [3, 20, 30]
}]

# データセット分割数を定義
cv = 5

# GridSearchCV クラスを用いたモデルの定義
tuned_model = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=cv, return_train_score=False)

# モデルの学習＆検証
tuned_model.fit(x_train_val, t_train_val)

# 検証結果の確認
pd.DataFrame(tuned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.005439,0.006448,0.005902,0.014741,0.014921,0.017882,0.021721,0.019807,0.020241
std_fit_time,0.002171,0.001549,0.000368,0.000905,0.000657,0.005634,0.000993,0.001126,0.001833
mean_score_time,0.000775,0.000955,0.001093,0.001638,0.001631,0.001353,0.001452,0.002122,0.001933
std_score_time,0.000117,0.000114,0.000164,0.000698,0.00068,0.000138,0.00012,0.001501,0.000874
param_max_depth,3,3,3,10,10,10,50,50,50
param_min_samples_split,3,20,30,3,20,30,3,20,30
params,"{'max_depth': 3, 'min_samples_split': 3}","{'max_depth': 3, 'min_samples_split': 20}","{'max_depth': 3, 'min_samples_split': 30}","{'max_depth': 10, 'min_samples_split': 3}","{'max_depth': 10, 'min_samples_split': 20}","{'max_depth': 10, 'min_samples_split': 30}","{'max_depth': 50, 'min_samples_split': 3}","{'max_depth': 50, 'min_samples_split': 20}","{'max_depth': 50, 'min_samples_split': 30}"
split0_test_score,0.88335,0.88335,0.88335,0.880359,0.881356,0.874377,0.836491,0.861416,0.865404
split1_test_score,0.889332,0.889332,0.889332,0.880359,0.886341,0.887338,0.825523,0.871386,0.879362
split2_test_score,0.890329,0.890329,0.890329,0.880359,0.88335,0.882353,0.837488,0.878365,0.877368


In [182]:
# GridSearchCV クラスのインポート
from sklearn.model_selection import GridSearchCV

from sklearn.tree import DecisionTreeClassifier

# 学習に使用するアルゴリズムの定義
estimator = DecisionTreeClassifier(random_state=0)

# 探索するハイパーパラメータと範囲の定義
param_grid = [{
    'max_depth': [3, 5, 50],
    'min_samples_split': [3, 20, 30]
}]

# データセット分割数を定義
cv = 5

# GridSearchCV クラスを用いたモデルの定義
tuned_model = GridSearchCV(estimator=estimator, 
                           param_grid=param_grid, 
                           cv=cv, return_train_score=False)

# モデルの学習＆検証
tuned_model.fit(x_train_val, t_train_val)

# 検証結果の確認
pd.DataFrame(tuned_model.cv_results_).T

,0,1,2,3,4,5,6,7,8
mean_fit_time,0.006127,0.006048,0.005855,0.008746,0.008889,0.00904,0.020811,0.019711,0.020769
std_fit_time,0.000201,0.000147,0.000156,0.000186,0.000171,0.000629,0.00073,0.000565,0.002223
mean_score_time,0.001406,0.001397,0.00101,0.001248,0.001206,0.001327,0.001388,0.001298,0.001364
std_score_time,0.000198,0.000689,0.000068,0.000075,0.000045,0.000375,0.000056,0.000051,0.00007
param_max_depth,3,3,3,5,5,5,50,50,50
param_min_samples_split,3,20,30,3,20,30,3,20,30
params,"{'max_depth': 3, 'min_samples_split': 3}","{'max_depth': 3, 'min_samples_split': 20}","{'max_depth': 3, 'min_samples_split': 30}","{'max_depth': 5, 'min_samples_split': 3}","{'max_depth': 5, 'min_samples_split': 20}","{'max_depth': 5, 'min_samples_split': 30}","{'max_depth': 50, 'min_samples_split': 3}","{'max_depth': 50, 'min_samples_split': 20}","{'max_depth': 50, 'min_samples_split': 30}"
split0_test_score,0.88335,0.88335,0.88335,0.879362,0.878365,0.876371,0.836491,0.861416,0.865404
split1_test_score,0.889332,0.889332,0.889332,0.891326,0.890329,0.890329,0.825523,0.871386,0.879362
split2_test_score,0.890329,0.890329,0.890329,0.887338,0.882353,0.88335,0.837488,0.878365,0.877368


In [183]:
# 最も予測精度の高かったモデルの引き継ぎ
best_model = tuned_model.best_estimator_

# モデルの検証
print(best_model.score(x_train_val, t_train_val))
print(best_model.score(x_test, t_test))

0.8920805904647915
0.8708133971291866


In [184]:
df_pred = df_pred.values
#推論
y_pred = best_model.predict(df_pred)

In [185]:
# 推論結果を、 success, fail に書き換える
y_list = np.where(y_pred, 'success', 'fail').tolist()

In [186]:
# Windows の方
pd.Series(y_list, name='pred').to_csv('submission.csv', index=None, line_terminator='\n')

In [187]:
df_all = pd.read_csv('submission.csv')
df_all.shape

(10000, 1)

In [188]:
df_pred.shape

(10000, 9)

In [189]:
pd.read_csv('submission.csv')

,pred
0,fail
1,fail
2,fail
3,fail
4,fail
...,...
9995,fail
9996,fail
9997,fail
9998,fail


In [190]:
print(df_train3.shape)
print(df_test.shape)
print(df_pred.shape)

(6267, 10)
(15000, 12)
(10000, 9)
